In [2]:
import torch
from transformers import BertTokenizer, BertForTokenClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased')


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
model.config.id2label = {0: "location", 1: "people name"}
text = "My name is Aman. I am a student."
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)


In [15]:
with torch.no_grad():
    outputs = model(**inputs)

predicted_labels_indices = torch.argmax(outputs.logits, dim=2)


In [19]:
# Convert token IDs to labels
label_names = [model.config.id2label[label_id.item()] for label_id in predicted_labels_indices]

# Print token and corresponding label name
decoded_tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
for token, label_name in zip(decoded_tokens, label_names):
    if '##' in token:
        # Handle subword tokens by appending them to the previous token
        print(f"Token: {decoded_tokens[-1]}{token.replace('##', '')}\tPredicted Label: {label_name}")
    else:
        print(f"Token: {token}\tPredicted Label: {label_name}")

RuntimeError: a Tensor with 12 elements cannot be converted to Scalar

In [22]:
from transformers import BertTokenizer, BertForTokenClassification

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased')

# Modify the id2label mapping in the model's configuration
model.config.id2label = {0: "location", 1: "people name",2:"organization"}  # Modify this mapping according to your labels

# Define an example input in Nepali
text = "मेरो नाम अमन हो। म एक विद्यार्थी हूँ।"

# Tokenize and encode input
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

# Run inference
with torch.no_grad():
    outputs = model(**inputs)

# Get predicted labels (integer indices)
predicted_label_indices = torch.argmax(outputs.logits, dim=2)[0]

# Convert label indices to label names using modified model's configuration
label_names = [model.config.id2label[label_id.item()] for label_id in predicted_label_indices]

# Print token and corresponding label name
decoded_tokens = []
for token, label_name in zip(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]), label_names):
    if '##' in token and len(decoded_tokens) > 0:
        decoded_tokens[-1] += token.replace('##', '')
    else:
        decoded_tokens.append(token.replace('##', ''))
    
for token, label_name in zip(decoded_tokens, label_names):
    print(f"Token: {token}\tPredicted Label: {label_name}")


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Token: [CLS]	Predicted Label: people name
Token: मेरो	Predicted Label: people name
Token: नाम	Predicted Label: people name
Token: अमन	Predicted Label: people name
Token: हो	Predicted Label: people name
Token: ।	Predicted Label: people name
Token: म	Predicted Label: people name
Token: एक	Predicted Label: people name
Token: विद्यार्थी	Predicted Label: people name
Token: हूँ	Predicted Label: people name
Token: ।	Predicted Label: people name
Token: [SEP]	Predicted Label: location
